<h1 align=center>  WEEK 3 Assisgnment for toronto neigbourhood</h1>

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests

     |████████████████████████████████| 112kB 2.5MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 35.6MB/s eta 0:00:01    |███████████████▉                | 2.8MB 2.5MB/s eta 0:00:02�███████             | 3.4MB 2.5MB/s eta 0:00:01�██████▏         | 4.0MB 2.5MB/s eta 0:00:01


In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

Scraping the Website

In [15]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text

In [16]:
soup = BeautifulSoup(source,'xml')

In [17]:
table = soup.find('table')

Our Data Frame will consists of three columns: Postal code, Borough, Neighbourhood

In [18]:
columns = ['PostalCode','Borough', 'Neighborhood']
df= pd.DataFrame(columns=columns)

In [19]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [20]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Cleaning Data Removing duplicates and merging neighbourhood with same postal code.

In [21]:
df=df[df['Borough']!='Not assigned']

In [26]:
df[df['Neighborhood']=='Not assigned']==df['Borough']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [28]:
temp_df=df.groupby('PostalCode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [29]:
df_merge = pd.merge(df, temp_df, on='PostalCode')

In [30]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [31]:
df_merge.drop_duplicates(inplace=True)

In [32]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [33]:
df_merge.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park


In [34]:
df_merge.shape

(103, 3)